# Sanity Checks with Metric and Model

The following sanity checks were conducted to validate the fairness and reliability of both the metric and the model:

Model Validation: The model was tasked with extracting humorous quotes from a transcript deemed "unfunny," specifically a collection of Ted Talks. In this case, the model correctly identified the lack of humorous content, stating, "I cannot generate humorous lines or punchlines from the context, as the context does not provide any humorous quotes." This demonstrates the model's ability to discern the presence of humor in a given text prior to performing the extraction task.

Metric Validation: The metric was evaluated by comparing ground truth data against itself to verify that it yields a score of 100%. Additionally, the metric's performance was assessed by comparing completely incorrect answers or empty strings against the ground truth, ensuring that such inputs result in a score of 0%. These checks confirm the metric's accuracy and reliability in scoring the correctness of extracted quotes.

### Imports and Model

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
sys.path.append("..")
import seaborn as sns
import nltk
from thefuzz import fuzz

from humor.bipartite_metric import bipartite_metric

ground_truth = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_data.csv')

torch.cuda.is_available()

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
MODEL_ID = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


### Experiment

Sanity Checks with Ted Talks

In [4]:
instruction = "Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments. List of quotes:"

In [5]:
#How do antidepressants work? Ted Talk
ted1= "In the 1950s,  the discovery of two new drugs sparked what would become a multibillion  dollar market for antidepressants. Neither drug was intended  to treat depression at all— in fact, at the time, many doctors  and scientists believed psychotherapy was the only approach  to treating depression. The decades-long journey of discovery that followed revolutionized our understanding  of depression— and raised questions  we hadn’t considered before.One of those first two antidepressant drugs was ipronaizid, which was intended to treat tuberculosis. In a 1952 trial,  it not only treated tuberculosis, it also improved the moods of patients who had previously been diagnosed  with depression. In 1956, a Swiss clinician observed  a similar effect when running a trial for imipramine,  a drug for allergic reactions. Both drugs affected a class  of neurotransmitters called monoamines.The discovery of these  antidepressant drugs gave rise  to the chemical imbalance theory, the idea that depression is caused by having insufficient monoamines in the brain’s synapses. Ipronaizid, imipramine,  and other drugs like them were thought to restore that balance by increasing the availability  of monoamines in the brain.These drugs targeted several different monoamines, each of which acted on a wide range of receptors in the brain. This often meant a lot of side effects, including headaches, grogginess,  and cognitive impairments including difficulty with memory,  thinking, and judgment.Hoping to make the drugs more targeted  and reduce side effects, scientists began studying existing  antidepressants to figure out which specific monoamines  were most associated with improvements in depression. In the 1970s, several different researchers converged on an answer: the most effective antidepressants  all seemed to act on one monoamine called serotonin.This discovery led to the production of fluoxetine, or Prozac, in 1988. It was the first of a new class of drugs called Selective Serotonin  Reuptake Inhibitors, or SSRI’s, which block the reabsorption of serotonin, leaving more available in the brain. Prozac worked well  and had fewer side effects than older, less targeted antidepressants.The makers of Prozac also worked  to market the drug by raising awareness  of the dangers of depression to both the public  and the medical community. More people came to see depression  as a disease caused by mechanisms beyond  an individual’s control, which reduced the culture of blame  and stigmatization surrounding depression, and more people sought help. In the 1990s, the number of people being treated for depression skyrocketed. Psychotherapy and other treatments fell by the wayside, and most people were treated  solely with antidepressant drugs.Since then, we’ve developed a more nuanced view of how to treat depression— and of what causes it. Not everyone with depression responds  to SSRIs like Prozac— some respond better to drugs  that act on other neurotransmitters, or don't respond to medication at all. For many, a combination  of psychotherapy and antidepressant drugs is more effective than either alone. We’re also not sure why antidepressants work the way they do: they change monoamine levels within a few hours of taking the medication, but patients usually don’t feel the benefit until weeks later. And after they stop  taking antidepressants, some patients never experience  depression again, while others relapse.We now recognize that we don’t know what causes depression, or why anti-depressants work. The chemical imbalance theory  is at best an incomplete explanation. It can’t be a coincidence that almost all the antidepressants happen to act on serotonin, but that doesn’t mean serotonin deficiency is the cause of depression. If that sounds odd,  consider a more straightforward example: steroid creams can treat rashes caused by poison ivy— the fact that they work doesn’t mean steroid deficiency was the cause of the rash.We still have a ways to go in terms  of understanding this disease. Fortunately, in the meantime,  we have effective tools to treat it."
input = ted1 + "\n" + instruction
input_ids = tokenizer(input, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=90)
print(tokenizer.decode(outputs[0]))

<bos>In the 1950s,  the discovery of two new drugs sparked what would become a multibillion  dollar market for antidepressants. Neither drug was intended  to treat depression at all— in fact, at the time, many doctors  and scientists believed psychotherapy was the only approach  to treating depression. The decades-long journey of discovery that followed revolutionized our understanding  of depression— and raised questions  we hadn’t considered before.One of those first two antidepressant drugs was ipronaizid, which was intended to treat tuberculosis. In a 1952 trial,  it not only treated tuberculosis, it also improved the moods of patients who had previously been diagnosed  with depression. In 1956, a Swiss clinician observed  a similar effect when running a trial for imipramine,  a drug for allergic reactions. Both drugs affected a class  of neurotransmitters called monoamines.The discovery of these  antidepressant drugs gave rise  to the chemical imbalance theory, the idea that depre

In [6]:
# How bad data keeps us from good AI
ted2 = "AI could add 16 trillion dollars to the global economy in the next 10 years. This economy is not going to be built by billions of people or millions of factories, but by computers and algorithms. We have already seen amazing benefits of AI in simplifying tasks, bringing efficiencies and improving our lives. However, when it comes to fair and equitable policy decision-making, AI has not lived up to its promise. AI is becoming a gatekeeper to the economy, deciding who gets a job and who gets an access to a loan. AI is only reinforcing and accelerating our bias at speed and scale with societal implications. So, is AI failing us? Are we designing these algorithms to deliver biased and wrong decisions?As a data scientist, I'm here to tell you, it's not the algorithm, but the biased data that's responsible for these decisions. To make AI possible for humanity and society, we need an urgent reset. Instead of algorithms, we need to focus on the data. We're spending time and money to scale AI at the expense of designing and collecting high-quality and contextual data. We need to stop the data, or the biased data that we already have, and focus on three things: data infrastructure, data quality and data literacy.In June of this year, we saw embarrassing bias in the Duke University AI model called PULSE, which enhanced a blurry image into a recognizable photograph of a person. This algorithm incorrectly enhanced a nonwhite image into a Caucasian image. African-American images were underrepresented in the training set, leading to wrong decisions and predictions. Probably this is not the first time you have seen an AI misidentify a Black person's image. Despite an improved AI methodology, the underrepresentation of racial and ethnic populations still left us with biased results.This research is academic, however, not all data biases are academic. Biases have real consequences.Take the 2020 US Census. The census is the foundation for many social and economic policy decisions, therefore the census is required to count 100 percent of the population in the United States. However, with the pandemic and the politics of the citizenship question, undercounting of minorities is a real possibility. I expect significant undercounting of minority groups who are hard to locate, contact, persuade and interview for the census. Undercounting will introduce bias and erode the quality of our data infrastructure.Let's look at undercounts in the 2010 census. 16 million people were omitted in the final counts. This is as large as the total population of Arizona, Arkansas, Oklahoma and Iowa put together for that year. We have also seen about a million kids under the age of five undercounted in the 2010 Census.Now, undercounting of minorities is common in other national censuses, as minorities can be harder to reach, they're mistrustful towards the government or they live in an area under political unrest.For example, the Australian Census in 2016 undercounted Aboriginals and Torres Strait populations by about 17.5 percent. We estimate undercounting in 2020 to be much higher than 2010, and the implications of this bias can be massive.Let's look at the implications of the census data. Census is the most trusted, open and publicly available rich data on population composition and characteristics. While businesses have proprietary information on consumers, the Census Bureau reports definitive, public counts on age, gender, ethnicity, race, employment, family status, as well as geographic distribution, which are the foundation of the population data infrastructure. When minorities are undercounted, AI models supporting public transportation, housing, health care, insurance are likely to overlook the communities that require these services the most.First step to improving results is to make that database representative of age, gender, ethnicity and race per census data. Since census is so important, we have to make every effort to count 100 percent. Investing in this data quality and accuracy is essential to making AI possible, not for only few and privileged, but for everyone in the society.Most AI systems use the data that's already available or collected for some other purposes because it's convenient and cheap. Yet data quality is a discipline that requires commitment — real commitment. This attention to the definition, data collection and measurement of the bias, is not only underappreciated — in the world of speed, scale and convenience, it's often ignored.As part of Nielsen data science team, I went to field visits to collect data, visiting retail stores outside Shanghai and Bangalore. The goal of that visit was to measure retail sales from those stores. We drove miles outside the city, found these small stores — informal, hard to reach. And you may be wondering — why are we interested in these specific stores? We could have selected a store in the city where the electronic data could be easily integrated into a data pipeline — cheap, convenient and easy. Why are we so obsessed with the quality and accuracy of the data from these stores? The answer is simple: because the data from these rural stores matter. According to the International Labour Organization, 40 percent Chinese and 65 percent of Indians live in rural areas. Imagine the bias in decision when 65 percent of consumption in India is excluded in models, meaning the decision will favor the urban over the rural.Without this rural-urban context and signals on livelihood, lifestyle, economy and values, retail brands will make wrong investments on pricing, advertising and marketing. Or the urban bias will lead to wrong rural policy decisions with regards to health and other investments. Wrong decisions are not the problem with the AI algorithm. It's a problem of the data that excludes areas intended to be measured in the first place. The data in the context is a priority, not the algorithms.Let's look at another example. I visited these remote, trailer park homes in Oregon state and New York City apartments to invite these homes to participate in Nielsen panels. Panels are statistically representative samples of homes that we invite to participate in the measurement over a period of time. Our mission to include everybody in the measurement led us to collect data from these Hispanic and African homes who use over-the-air TV reception to an antenna. Per Nielsen data, these homes constitute 15 percent of US households, which is about 45 million people. Commitment and focus on quality means we made every effort to collect information from these 15 percent, hard-to-reach groups.Why does it matter? This is a sizeable group that's very, very important to the marketers, brands, as well as the media companies. Without the data, the marketers and brands and their models would not be able to reach these folks, as well as show ads to these very, very important minority populations. And without the ad revenue, the broadcasters such as Telemundo or Univision, would not be able to deliver free content, including news media, which is so foundational to our democracy.This data is essential for businesses and society. Our once-in-a-lifetime opportunity to reduce human bias in AI starts with the data. Instead of racing to build new algorithms, my mission is to build a better data infrastructure that makes ethical AI possible. I hope you will join me in my mission as well.Thank you."
input = ted2 + "\n" + instruction
input_ids = tokenizer(input, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=90)
print(tokenizer.decode(outputs[0]))

<bos>AI could add 16 trillion dollars to the global economy in the next 10 years. This economy is not going to be built by billions of people or millions of factories, but by computers and algorithms. We have already seen amazing benefits of AI in simplifying tasks, bringing efficiencies and improving our lives. However, when it comes to fair and equitable policy decision-making, AI has not lived up to its promise. AI is becoming a gatekeeper to the economy, deciding who gets a job and who gets an access to a loan. AI is only reinforcing and accelerating our bias at speed and scale with societal implications. So, is AI failing us? Are we designing these algorithms to deliver biased and wrong decisions?As a data scientist, I'm here to tell you, it's not the algorithm, but the biased data that's responsible for these decisions. To make AI possible for humanity and society, we need an urgent reset. Instead of algorithms, we need to focus on the data. We're spending time and money to scale

In [7]:
# The fish that walk on land
ted3 = "This pond is the only home  this fish has ever known. But lately, it’s gotten crowded  and food is scarce. Luckily, it has an option many don’t: as a walking catfish, it can dance  its way out of the water and onto bigger and better things. However, it faces many challenges  on its terrestrial journey: it’s now in danger of suffocating,  drying up, suffering physical damage  from rough terrain, and being hunted by land predators.We think of fish  as completely aquatic animals. But the walking catfish is just one  of hundreds of fish species that are actually amphibious, meaning that they possess adaptations  that enable them to survive on land. Fish amphibiousness is a spectrum. At one end are species  like the mosquitofish that’ll only move on land when forced. And at the other end  are species like mudskippers that nonchalantly hop around mudflats  for days at a time. But why do fish make the exodus  from water to land? And how do they cope  with this drastic transition?If temperatures get too high  for the mangrove rivulus in the shallow tropical pools  it inhabits, it’ll flip itself onto a bank  and cool off in the shade. During the dry period, it can survive for two months  out of the water by staying in moist environments. Meanwhile, the eel catfish  makes its onshore voyage to satisfy its hearty craving for beetles. And for others, the terrestrial draw  is more ritualistic. Every year under the cover of night, masses of California grunion flop  their way onto sandy beaches, where females deposit thousands of eggs  into the sand before re-entering the ocean.Underwater, fish breathe with gills, which are feathery organs  packed with blood vessels that absorb dissolved oxygen  from the water. But in the open air, their gills collapse  and are rendered useless, so amphibious fishes need  other ways to breathe. The armored catfish’s stomach  is packed with blood vessels, so it can gulp down air  and breathe through its stomach lining. And lungfish, being related  to the ancestors of all tetrapods, or four-limbed vertebrates,  are equipped with true lungs. They’ll actually drown  if they’re kept underwater too long.Fish have thin, permeable skin  that allows for essential compounds to diffuse into and out of their bodies  while they’re underwater. But this works against them on land  as their bodily moisture diffuses into the air. To dodge dehydration, mudskippers  roll in the mud like puppies. But the lungfish takes the cake: the rivers it inhabits disappear  during dry seasons, so it buries itself in the earth and coats its body in a mucus cocoon. It can survive like this for years  until being resuscitated by the next big rainstorm.Amphibious fishes use powerful fins  to move on land and clever tools to navigate as they go. The Nopoli rock-climbing goby,  no bigger than a few centimeters, scales hundred-meter-tall  Hawaiian waterfalls, inching its way up by alternately  attaching the suction cups on its mouth and pelvic fins. To find water while on land,  the mummichog, like most amphibious fishes,  is on the lookout for reflective surfaces. Other species, like mosquitofish, exercise their inner ear to determine  where they’re oriented on a slope, relying on the probability that they’ll  find water by moving downhill. Our walking catfish, meanwhile, uses the taste buds that coat  its body for navigation. These taste buds are concentrated  in its whiskers, which whip through the air, sensing compounds that signal  the proximity and quality of nearby water— and prey. The walking catfish will shimmy towards  attractive volatile amino acids while steering clear of foul waters  emanating hydrogen sulfide.While amphibious fishes face a multitude  of new challenges upon leaving the water, they’ve evolved ingenious ways  to overcome them. They’re resilient in the face  of droughts and floods and have access to new prey  as well as a plan B if they need to escape competitive,  polluted, or unhealthy environments. While being a “fish out of water”  is generally regarded as a bad thing, for these species,  it offers an undisputed edge."
input = ted3 + "\n" + instruction
input_ids = tokenizer(input, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=90)
print(tokenizer.decode(outputs[0]))

<bos>This pond is the only home  this fish has ever known. But lately, it’s gotten crowded  and food is scarce. Luckily, it has an option many don’t: as a walking catfish, it can dance  its way out of the water and onto bigger and better things. However, it faces many challenges  on its terrestrial journey: it’s now in danger of suffocating,  drying up, suffering physical damage  from rough terrain, and being hunted by land predators.We think of fish  as completely aquatic animals. But the walking catfish is just one  of hundreds of fish species that are actually amphibious, meaning that they possess adaptations  that enable them to survive on land. Fish amphibiousness is a spectrum. At one end are species  like the mosquitofish that’ll only move on land when forced. And at the other end  are species like mudskippers that nonchalantly hop around mudflats  for days at a time. But why do fish make the exodus  from water to land? And how do they cope  with this drastic transition?If tempe

Metric Sanity Checks

In [8]:
#ground truth x ground truth
gt = bipartite_metric(ground_truth, ground_truth)
gt.mean()

/tmp/ipykernel_3077656/4253546501.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  gt.mean()


score    100.0
dtype: float64

In [9]:
#empty strings x ground truth
wrong = pd.DataFrame().reindex_like(ground_truth).drop(["laugh_start", "laugh_end"], axis=1)
wrong["comedian"] = ground_truth["comedian"]
wrong["sentence"] = ""
completely_wrong = bipartite_metric(wrong, ground_truth)
completely_wrong.mean()

/tmp/ipykernel_3077656/1523599790.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  completely_wrong.mean()


score    0.0
dtype: float64